In [1]:
## Import packages to read in data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

In [2]:
## Data does not have column headers, add these when importing

column_labels = ['transaction_ID', 'price', 'transfer_date', 'postcode','property_type', 
              'old_vs_new','duration', 'PAON', 'SAON', 'street', 'locality', 'town_city', 
              'district', 'county', 'PPD_category_type', 'record_status']

## Read in the UK Housing Prices Paid dataset
## requires chunking as it is a large file

df_chunk = pd.read_csv("/Users/user/Desktop/Datasets/pp-2019-part1.csv", names=column_labels,
                      iterator=True, chunksize=10000)

## concatenate the chunks into one DataFrame 
chunk_list = []  # append each chunk here 

# Each chunk is in df format
for chunk in df_chunk:      
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
df = pd.concat(chunk_list)

In [3]:
## Convert transfer date to DateTime object
df['transfer_date'] = pd.to_datetime(df.transfer_date)

## Inspect the dataset by examining details
type(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179336 entries, 0 to 179335
Data columns (total 16 columns):
transaction_ID       179336 non-null object
price                179336 non-null int64
transfer_date        179336 non-null datetime64[ns]
postcode             178138 non-null object
property_type        179336 non-null object
old_vs_new           179336 non-null object
duration             179336 non-null object
PAON                 179336 non-null object
SAON                 22563 non-null object
street               176240 non-null object
locality             67380 non-null object
town_city            179336 non-null object
district             179336 non-null object
county               179336 non-null object
PPD_category_type    179336 non-null object
record_status        179336 non-null object
dtypes: datetime64[ns](1), int64(1), object(14)
memory usage: 21.9+ MB


In [4]:
## Add month and year columns
df['month'] = df['transfer_date'].dt.month
df['year'] = df['transfer_date'].dt.year

df[(df.price > 600000) & (df.property_type == 'F')]

,transaction_ID,price,transfer_date,postcode,property_type,old_vs_new,duration,PAON,SAON,street,locality,town_city,district,county,PPD_category_type,record_status,month,year
170,{80E1AA98-B729-7BF8-E053-6C04A8C00BF2},625000,2019-01-09,SW19 7DR,F,N,L,119,FLAT 1,ARTHUR ROAD,NaN,LONDON,MERTON,GREATER LONDON,A,A,1,2019
302,{8A78B2AF-3494-5CB0-E053-6B04A8C0F504},705000,2019-03-11,N4 2BX,F,Y,L,"SKYLARK POINT, 48",FLAT 29,NEWNTON CLOSE,NaN,LONDON,HACKNEY,GREATER LONDON,A,A,3,2019
320,{8A78B2AF-34AB-5CB0-E053-6B04A8C0F504},1380000,2019-02-08,E1 8ZG,F,Y,L,"PERILLA HOUSE, 17",FLAT 1901,STABLE WALK,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,2,2019
329,{8A78B2AF-34B4-5CB0-E053-6B04A8C0F504},635000,2019-03-28,W3 8UU,F,N,L,MUNSTER COURT,49,BOLLO BRIDGE ROAD,NaN,LONDON,EALING,GREATER LONDON,A,A,3,2019
332,{8A78B2AF-3504-5CB0-E053-6B04A8C0F504},740000,2019-04-15,N4 2GX,F,Y,L,"SANDPIPER BUILDING, 44",FLAT 19,NEWNTON CLOSE,NaN,LONDON,HACKNEY,GREATER LONDON,A,A,4,2019
334,{8A78B2AF-3506-5CB0-E053-6B04A8C0F504},618000,2019-04-01,E1 8QW,F,Y,L,"EASTLIGHT APARTMENTS, 18",FLAT 405,DOCK STREET,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,4,2019
339,{8A78B2AF-350F-5CB0-E053-6B04A8C0F504},685000,2019-03-28,E1 8QU,F,Y,L,"LUXE TOWER, 12",FLAT 1003,DOCK STREET,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,3,2019
342,{8A78B2AF-3513-5CB0-E053-6B04A8C0F504},838000,2019-03-25,E1 8QU,F,Y,L,"LUXE TOWER, 12",FLAT 402,DOCK STREET,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,3,2019
352,{8A78B2AF-3522-5CB0-E053-6B04A8C0F504},808000,2019-04-01,E1 8QW,F,Y,L,"EASTLIGHT APARTMENTS, 18",FLAT 502,DOCK STREET,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,4,2019
355,{8A78B2AF-3526-5CB0-E053-6B04A8C0F504},675000,2019-04-01,HA1 1GY,F,Y,L,FLAT 93 BRYANT APARTMENTS,NaN,PERCEVAL SQUARE,NaN,HARROW,HARROW,GREATER LONDON,A,A,4,2019


In [5]:
## Splice data to search based on specified city
city_select = 'London'
city_table = df[df.town_city == city_select.upper()]

## Count number of transactions
print(type(city_table))

## Average price paid
avg_price_paid = round(city_table.price.mean(), 0)
print(avg_price_paid)
#print("Average price paid in " + city_select + " is: " + avg_price_paid.astype(int).astype(str))

## Print sorted table
city_table.head()

<class 'pandas.core.frame.DataFrame'>
1079849.0


,transaction_ID,price,transfer_date,postcode,property_type,old_vs_new,duration,PAON,SAON,street,locality,town_city,district,county,PPD_category_type,record_status,month,year
166,{80E1AA98-B71D-7BF8-E053-6C04A8C00BF2},755000,2019-01-23,SE21 8SY,T,N,F,16,NaN,LINGS COPPICE,NaN,LONDON,LAMBETH,GREATER LONDON,A,A,1,2019
168,{80E1AA98-B721-7BF8-E053-6C04A8C00BF2},350000,2019-01-04,SW17 0LB,F,N,L,49,NaN,SIWARD ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A,1,2019
169,{80E1AA98-B728-7BF8-E053-6C04A8C00BF2},690000,2019-01-18,SE6 1ND,T,N,F,35,NaN,BROADFIELD ROAD,NaN,LONDON,LEWISHAM,GREATER LONDON,A,A,1,2019
170,{80E1AA98-B729-7BF8-E053-6C04A8C00BF2},625000,2019-01-09,SW19 7DR,F,N,L,119,FLAT 1,ARTHUR ROAD,NaN,LONDON,MERTON,GREATER LONDON,A,A,1,2019
172,{80E1AA98-B72C-7BF8-E053-6C04A8C00BF2},475000,2019-01-18,SE1 3AR,T,N,F,64,NaN,HENLEY DRIVE,NaN,LONDON,SOUTHWARK,GREATER LONDON,A,A,1,2019


In [6]:
## Summary statistics of dataset
property_type = 'F'
avg_price = np.mean(df.price)
median_price = np.median(df.price)

print(avg_price)
print(median_price)

341581.20966788597
222500.0


In [ ]:
London = df[df['town_city'] == 'LONDON']
Brighton = df[df['town_city'] == 'BRIGHTON']

plt.boxplot([London.price, Brighton.price])
plt.show()